### VGG 16 from scratch (The simplest way)

In [1]:
#importing required libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [6]:
class VGG16(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG16, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)
        
        self.conv2 = nn.Conv2d(64,64, kernel_size=3, padding=1)
        self.maxPool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(128,128, kernel_size=3, padding=1)
        #self.bn2 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128,128, kernel_size=3, padding=1)
        
        self.conv5 = nn.Conv2d(256,256, kernel_size=3, padding=1)
        #self.bn3 = nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(256,256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256,256, kernel_size=1, padding=0)
        
        self.conv8 = nn.Conv2d(512,512, kernel_size=3, padding=1)
        #self.bn4 = nn.BatchNorm2d(512)
        
        self.conv9 = nn.Conv2d(512,512, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(512,512, kernel_size=1, padding=0)
        
        self.conv11 = nn.Conv2d(512,512, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(512,512, kernel_size=3, padding=1)
        self.conv13 = nn.Conv2d(512,512, kernel_size=1, padding=0)
        
        self.linear1 = nn.Linear(512*7*7, 4096)
        self.linear2 = nn.Linear(4096, 4096)
        self.linear3 = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        out= self.conv1(x)
        out = self.conv2(out)
        out = self.maxPool(out)
        
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxPool(out)
        
        out = self.conv5(out)
        out = self.conv6(out)
        out = self.conv7(out)
        out = self.maxPool(out)
        
        out = self.conv8(out)
        out = self.conv9(out)
        out = self.conv10(out)
        out = self.maxPool(out)
        
        out = self.conv11(out)
        out = self.conv12(out)
        out = self.conv13(out)
        
        out = self.maxPool(out)
        
        out = out.view(out.size(0),-1)
        out = self.linear1(out)
        out = self.linear2(out)
        out = self.linear3(out)
        
        return out
        
        
        

In [13]:
model=VGG16()

In [14]:
model

VGG16(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxPool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (conv8): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv9): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv10): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
  (conv11): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv12): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding

In [10]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])



dataset_train = CIFAR10(root='../data', train=True, 
            download=True, transform=transform)
dataset_test = CIFAR10(root='../data', train=False, 
             download=True, transform=transform)
train_loader = DataLoader(dataset_train, batch_size=32, 
                        shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=32, 
                       shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)

for i, (input_,target) in enumerate(train_loader):
    target = target.to(device)
    input_ = input_.to(device)
    print(target.shape, input_.shape)

    output = model(input_)
    loss = criterion(output,target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i%20 == 0):
        print("loss in epoch %d , step %d : %f" % (epoch, i,loss.data[0]))

RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 2.00 GiB total capacity; 578.49 MiB already allocated; 133.06 MiB free; 21.51 MiB cached)